In [7]:
!pip3 install tf-models-official

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# Load YOLO model
yolo_net = cv2.dnn.readNetFromDarknet('yolov3.cfg', 'yolov3.weights')

# Load the COCO class labels
with open('coco.names', 'r') as f:
    classes = f.read().strip().split('\n')

# Get the output layer names
# Get the output layer names
layer_names = yolo_net.getLayerNames()
output_layers = [layer_names[yolo_net.getUnconnectedOutLayers()[0] - 1]]
# Open a video capture
cap = cv2.VideoCapture(r'WhatsApp Video 2024-10-11 at 2.08.29 PM.mp4')  # Change to a file path if needed

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Prepare the image for YOLO
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    yolo_net.setInput(blob)

    # Perform detection
    detections = yolo_net.forward(output_layers)

    boxes = []
    confidences = []
    class_ids = []

    # Process the detections
    for output in detections:
        for detection in output:
            scores = detection[5:]  # Get the scores for all classes
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Confidence threshold
                center_x = int(detection[0] * frame.shape[1])
                center_y = int(detection[1] * frame.shape[0])
                w = int(detection[2] * frame.shape[1])
                h = int(detection[3] * frame.shape[0])

                # Calculate the coordinates for the bounding box
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply non-maxima suppression to reduce overlapping boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Check if indices is not empty
    if len(indices) > 0:
        # Draw bounding boxes and labels on the frame
        for i in indices:
            box = boxes[i]
            (x, y, w, h) = box
            label = f"{classes[class_ids[i]]}: {confidences[i]:.2f}"
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Show the output
    cv2_imshow(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()